In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from trading.operations import create_trading_app, setup_trading_app, place_trade, cancel_trade, accept_trade, close_trading
from store.operations import StoringPool
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')

ALGOD_URL = os.environ.get('ALGOD_URL')
ALGOD_TOKEN = os.environ.get('ALGOD_TOKEN')
print(f"ALGOD_URL: {ALGOD_URL}")
print(f"ALGOD_TOKEN: {ALGOD_TOKEN}")

client = get_algod_client(ALGOD_URL, ALGOD_TOKEN)

ALGOD_URL: http://localhost:4001
ALGOD_TOKEN: aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa


In [3]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

storeAppID = int(os.environ.get("STORE_APP_ID"))
print(f"storeAppID: {storeAppID}")
#close_trading(client, 693, storeAppID, creator)

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

storeAppID: 4244
Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Bidder address: X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Seller Address: CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA


Alice is generating an example token...

In [4]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

# asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")

The token id is: 2989


In [5]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [6]:
storeApp = StoringPool(client=client, creator=creator)

storeAppID = int(os.environ.get("STORE_APP_ID"))
storeApp.app_id = storeAppID

# storeApp.create_app()
# storeAppID = storeApp.app_id
print(f"Store App ID: {storeAppID}")

storeAppAddress = get_application_address(storeAppID)
print(f"Store App Address: {storeAppAddress}")


Store App ID: 4244
Store App Address: BQPPZ7REPSFTB24WHGMNYZT34PCXD4FFG7BHNHCKXY6PILHQ5S7Z5KQAEQ


In [7]:

appID = create_trading_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction F7HIUDLSZ5FBZELDA76BBGRFMAGXKIP44AL3F466LEILS5KLXGAA confirmed in round 179496.
Waiting for confirmation...
Transaction 2IIXODP3NYJKXWZO6O4NCAF6CDCTXVSHKUTPY2LIT6FLAE3K2EGQ confirmed in round 179498.
App ID: 4296
App Address: MJK5OKGZRDWTVGDJSUMGZGJ63O3XDLBHB32PPTF4SC3Q4HGAD72X3HWVRY


In [8]:
storeApp.set_up(appID, 1234, 1233)

AlgodHTTPError: TransactionPool.Remember: transaction EXSC7IA4QGTKYKGHR44QRLRJUPGFYGJQ2R5EOEBQWWK4MYHYLOOQ: only clearing out is supported for applications that do not exist

Alice is setting up and funding token auction...

In [9]:
setup_trading_app(
    client=client,
    app_id=appID,
    funder=creator,
    token_id=asset_id,
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]
print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Waiting for confirmation...
Transaction E2JKE3BIML5YWQKYEHKK3K2IRBEMVNKTYJ62R2PFLSW7Z53N7CKQ confirmed in round 179501.
Alice's algo balance:  20088217211449  algos


In [14]:
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 201000, 2989: 0}


In [11]:
bidPrice = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidPrice, " algos")

Carla wants to bid on token, her algo balance:  10042137550982  algos
Carla is placing bid for:  1000000  algos


In [12]:
trading_index = place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice, trading_index="")

print("Carla bid", trading_index)

store_app_id 4244
new address: KO47IVZTQ34ECW5GUWJ3LYKFTYYD64EODFLHAUPIASWYEKOC5DH3ON7DOU
new private_key: qpGsOvbn16874GgLEm6lIaeqGC0fjnSlAypaQW3GPHRTufRXM4b4QVumpZO14UWeMD9wjhlWcFHoBK2CKcLozw==
new passphrase: crystal rather bubble legal quit typical achieve happy door reunion sport improve fetch economy west mix purse attend bench between surround cover peanut about scene
Waiting for confirmation...
Transaction LPHBWPEP47B3XF2WSILPWVA3IJV2E35QJUOGUYJEBOJOOXZYPAKA confirmed in round 179516.
Waiting for confirmation...
Transaction ROEO6PFYBQX7XCMAWGDMMMMUYGVD54FGRMIAPOMCJR26BLWAB72A confirmed in round 179518.
Waiting for confirmation...
Transaction BCHWCGWXNARRJHYLFWQ4HWSZZAGBBQSHHWYOZFQHDBBB4ZXU7FYA confirmed in round 179520.
token_txn: {'sender': 'CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA', 'fee': 506000, 'first_valid_round': 179514, 'last_valid_round': 180514, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'S46yr60YjyJzK3Fk0J8V7mzMIOSxovaXAVqCHlwPl1A=', 

In [13]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

cancel_trade(client, appID, seller, trading_index)
print("Carla cancel trading", trading_index)

Waiting for confirmation...
Transaction UVYYBWBV4C3RWLOLOCYMYMVZQKRGYPJBF7XRJ3TWZKQSBHRKYOUA confirmed in round 179526.
Carla cancel trading KO47IVZTQ34ECW5GUWJ3LYKFTYYD64EODFLHAUPIASWYEKOC5DH3ON7DOU


In [ ]:
trading_index = place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice, trading_index="")

place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=15, price=bidPrice, trading_index=trading_index)

In [ ]:
accept_trade(client, appID, bidder, seller.get_address(), trading_index)

Alice is closing out the auction....

In [ ]:
sleep(5)

close_trading(client, appID, creator, [asset_id])

In [ ]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

In [ ]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")

actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidPrice - 1_000
assert actualSellerBalances[asset_id] == 0

In [ ]:
app_global_state = get_app_global_state(client, storeAppID)
print(app_global_state)
seller_app_local_state = get_app_local_state(client, storeAppID, seller.get_address())
print(seller_app_local_state)
bidder_app_local_state = get_app_local_state(client, storeAppID, bidder.get_address())
print(bidder_app_local_state)
# print(encoding.encode_address(app_global_state[b"AS"]))
# print(encoding.encode_address(app_global_state[b"S"]))
#print(encoding.encode_address(storeAppState[b"C"]))